<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [805 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,690 kB]
Get:13 http://security.ubuntu.com/ubunt

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)


from bs4 import BeautifulSoup

In [3]:
# YouTube video URL
video_url = "https://www.youtube.com/watch?v=cZlsZwcIgpc"

In [ ]:
try:
    wait = WebDriverWait(driver, 5)

    # Open the YouTube video
    driver.get(video_url)

    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

    # Scroll down to load comments
    last_height = 0
    while True:
        # Scroll to the end of the page
        #driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        # Scroll using page-manager element's scroll height
        driver.execute_script("window.scrollTo(0, document.getElementById('page-manager').scrollHeight);")
        time.sleep(3)  # Adjust sleep time as needed

        # Get the current page height
        new_height = driver.execute_script("return document.getElementById('page-manager').scrollHeight")

        # Break the loop if no more content is loaded
        if new_height == last_height:
            break

        last_height = new_height

    # Wait for the comments section to be visible
    comments_section = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ytd-comments#comments.style-scope.ytd-watch-flexy")))

    # Get the HTML content of the comments section
    comments_html = comments_section.get_attribute('outerHTML')

finally:
    # Close the WebDriver session
    driver.quit()



In [ ]:
soup = BeautifulSoup(comments_html, 'html.parser')

# Find the span element with the specified class
comment_count_span = soup.find('span', class_='style-scope yt-formatted-string')

# Extract the text content of the span element
comment_count = comment_count_span.text.strip()

# Print or use the comment count
print("Comment Count:", comment_count)

In [ ]:
# Find all occurrences of the ytd-comment-thread-renderer element
comment_thread_renderers = soup.find_all('ytd-comment-thread-renderer', class_='style-scope ytd-item-section-renderer')

# Count the number of occurrences
comment_thread_count = len(comment_thread_renderers)

# Print or use the comment thread count
print("Number of ytd-comment-thread-renderer elements:", comment_thread_count)

In [ ]:
comments = []
# Iterate through each comment thread renderer
for comment_thread_renderer in comment_thread_renderers:

    # Extracting the comment text
    comment_text_element = comment_thread_renderer.find('yt-attributed-string', id='content-text')
    comment_text = comment_text_element.get_text(strip=True) if comment_text_element else None

    # Extracting the number of likes
    like_count_element = comment_thread_renderer.find('span', class_='style-scope ytd-comment-engagement-bar')
    like_count = like_count_element.get_text(strip=True) if like_count_element else None

    # Extracting the number of replies
    reply_count_element = comment_thread_renderer.find('ytd-button-renderer', id='more-replies')
    reply_count = reply_count_element.get_text(strip=True) if reply_count_element else None

    comments.append(comment_text)

    print("Comment Text:", comment_text)
    print("Like Count:", like_count)
    print("Reply Count:", reply_count)

    print("\n\n\n")


In [ ]:
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

In [ ]:
import googleapiclient.discovery
import datetime

import matplotlib.pyplot as plt
import matplotlib.image as mpimg



# Extract video ID from URL
video_id = video_url.split('=')[1]


print("Video ID:", video_id)

# Create YouTube API client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=video_id
).execute()

In [ ]:
# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=video_id
).execute()

# Extract required information
video = response["items"][0]
title = video["snippet"]["title"]
thumbnail_url = video["snippet"]["thumbnails"]["maxres"]["url"]
view_count = video["statistics"]["viewCount"]
like_count = video["statistics"]["likeCount"]
commentCount = video["statistics"]["commentCount"]
date_posted = video["snippet"]["publishedAt"]
description = video["snippet"]["description"]


date_str = date_posted
date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

# Format the date object
date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


# Split the description by newlines
paragraphs = description.split("\n")

# Get the first paragraph
first_paragraph = paragraphs[0]

# Generate HTML to display the thumbnail
thumbnail_html = f'<img src="{thumbnail_url}" alt="Thumbnail">'

# Print the information
print("Title:", title)
print("Thumbnail URL:", thumbnail_html)
print("View count:", view_count)
print("Like count:", like_count)
print("Number of Comments:", commentCount)
print("Date posted:", date_posted)
print("Description:", first_paragraph)


In [ ]:
from textblob import TextBlob

# Define a function to analyze the sentiment of a comment
def analyze_sentiment(comment):
    # Create a TextBlob object from the comment
    blob = TextBlob(comment)

    # Get the sentiment polarity of the comment
    polarity = blob.sentiment.polarity

    # Classify the sentiment based on the polarity
    if polarity > 0:
        sentiment = "positive"
    elif polarity < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    # Return the sentiment
    return sentiment

# Initialize a list to store the sentiments
sentiments = []

# Iterate through the comments and analyze their sentiment
for comment in comments:
    sentiment = analyze_sentiment(comment)
    sentiments.append(sentiment)

# Print the sentiments
for i, sentiment in enumerate(sentiments):
    print(f"Comment {i+1}: {sentiment} \n Comment: {comments[i]}")

# Count the number of positive, negative, and neutral comments
positive_count = sentiments.count("positive")
negative_count = sentiments.count("negative")
neutral_count = sentiments.count("neutral")

# Print the counts
print(f"Positive comments: {positive_count}")
print(f"Negative comments: {negative_count}")
print(f"Neutral comments: {neutral_count}")


In [ ]:
# Create a bar chart to visualize the sentiment analysis results
labels = ['Positive', 'Negative', 'Neutral']
values = [positive_count, negative_count, neutral_count]
colors = ['g', 'r', 'b']

plt.bar(labels, values, color=colors)
plt.title('Sentiment Analysis of Comments')
plt.xlabel('Sentiment')
plt.ylabel('Number of Comments')
plt.show()

# Calculate the percentage of each sentiment
total_comments = len(sentiments)
positive_percentage = (positive_count / total_comments) * 100
negative_percentage = (negative_count / total_comments) * 100
neutral_percentage = (neutral_count / total_comments) * 100

# Print the percentages
print(f"Positive comments: {positive_percentage:.2f}%")
print(f"Negative comments: {negative_percentage:.2f}%")
print(f"Neutral comments: {neutral_percentage:.2f}%")


In [ ]:
# Define the topic
topic = "Machine Learning"

# Search for videos related to the topic
response = youtube.search().list(
    q=topic,
    type="video",
    part="id,snippet",
    maxResults=10
).execute()

# Extract video IDs and titles
video_ids = []
video_titles = []
for item in response["items"]:
    video_ids.append(item["id"]["videoId"])
    video_titles.append(item["snippet"]["title"])

# Print the video links
for i, video_id in enumerate(video_ids):
    print(f"{i+1}. https://www.youtube.com/watch?v={video_id}")